In [11]:
import tensorflow as tf
import pickle
import read_data

In [20]:
def display_number_of_parameters():
    total_parameters = 0
    for variable in tf.trainable_variables():  
    	local_parameters=1
    	shape = variable.get_shape()  #getting shape of a variable
    	for i in shape:
        	local_parameters*=i.value  #mutiplying dimension values
    	total_parameters+=local_parameters
    print("Total number of hidden parameters: %i" % total_parameters)

In [ ]:
model_path = ".\\cifar_model_one"
# model_path = ".\\cifar_model_two"

In [24]:
num_of_samples = 4
top_n_predictions = 5
batch_size = 64

def test_model():
    test_images, test_labels = pickle.load(open('preprocess_training.p', mode='rb'))
    loaded_graph = tf.Graph()
    
    with tf.Session(graph=loaded_graph) as sess:
        #Load model
        loader = tf.train.import_meta_graph(model_path + '.meta')
        loader.restore(sess, model_path)
        
        #Get tensors from loaded model
        loaded_images = loaded_graph.get_tensor_by_name('image:0')
        loaded_labels = loaded_graph.get_tensor_by_name('label:0')
        loaded_accuarcy = loaded_graph.get_tensor_by_name('accuracy:0')
        
        display_number_of_parameters()
        test_batch_accuracy_total = 0
        test_batch_count = 0
        
        for train_image_batch, train_label_batch in read_data.batch_images_labels(test_images, test_labels, batch_size):
            test_batch_accuracy_total += sess.run(loaded_accuarcy, feed_dict={loaded_images:train_image_batch, loaded_labels:train_label_batch})
            test_batch_count += 1
        print('Testing Accuracy: {}\n'.format(test_batch_accuracy_total / test_batch_count))

In [25]:
test_model()

INFO:tensorflow:Restoring parameters from .\cifar_model_one


Total number of parameters: 273674


Testing Accuracy: 0.2818471337579618



In [ ]:
def mcnemar_test():
    